# 主程式

In [441]:
# update anytime
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [442]:
# 讀取套件與函數
from t0_config import *
from library.apply import *
from transformers import pipeline
from docx import Document
import os
os.chdir(doc_path)
tqdm.pandas()

In [443]:
# 讀取Word檔案
doc = Document(file_name + '.docx')
raw_text_list = [paragraph.text for paragraph in doc.paragraphs]
target_text_list = translation_filter(raw_text_list, target_range)

即將翻譯整篇文章共 91 句
從以下開始翻譯: 车辆格栅结构


In [444]:
# 文字處理
df = pd.DataFrame({
	'sentence':[ [text] for text in target_text_list],
	'skip_train':[skip_train(text) for text in target_text_list],
})

df['split_sentence'] = (df['sentence']
	.apply(lambda x: split_string_by_dot(x))
	.apply(lambda x: split_string_by_semicolon(x))
	.apply(lambda x: split_string_by_comma(x))
)

max_times = 0
max_of_maxlength = count_max_characters(df['split_sentence'])
while (max_of_maxlength > 100) & (max_times < 5):
	df['split_sentence'] = df['split_sentence'].apply(lambda x: split_string_by_comma(x))
	max_of_maxlength = count_max_characters(df['split_sentence'])
	max_times += 1


In [445]:
# 讀取機器學習模型
translator = pipeline("translation", model=model_checkpoint)

model.safetensors: 100%|██████████| 310M/310M [00:24<00:00, 12.9MB/s] 


In [470]:
df.head(10)

,sentence,skip_train,split_sentence
0,[车辆格栅结构],False,[车辆格栅结构]
1,[],True,[]
2,[],True,[]
3,[Technical Field],True,[Technical Field]
4,[本发明涉及一种车辆结构，尤其涉及一种车辆格栅（grill）结构。],False,"[本发明涉及一种车辆结构，尤其涉及一种车辆格栅（grill）结构。, ]"
5,[Description of Related Art],False,[Description of Related Art]
6,[在现有技术中，车辆格栅结构包括具有格栅开口的车辆外装部件（例如，保险杠）、以及设置在格栅开...,False,[在现有技术中，车辆格栅结构包括具有格栅开口的车辆外装部件（例如，保险杠）、以及设置在格栅开...
7,[],True,[]
8,[SUMMARY],True,[SUMMARY]
9,[本发明提供一种车辆格栅结构，能够抑制风噪音且能够兼顾外观性。],False,"[本发明提供一种车辆格栅结构，能够抑制风噪音且能够兼顾外观性。, ]"


In [471]:
# 開始翻譯
def translate_sentence(row:pd.Series)->str:
    if row['skip_train']:
        return ''.join(row['split_sentence'])

    paragraph = ''
    for sentence in row['split_sentence']:
        print(sentence)
        if not sentence.strip():
            print('continue')
            continue
        paragraph += translator(sentence, max_length=1000, return_text=True)[0]['translation_text']
    return paragraph

trans_text_list = df.progress_apply(translate_sentence, axis=1)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:24<00:00,  2.46s/it]


In [476]:
trans_text_list[9]

'The invention provides a vehicle grid structure capable of suppressing wind noise and accommodating appearance.'

In [447]:
# 產生新的Word檔案
for i in trange(len(trans_text_list)):
    paragraph = doc.paragraphs[i]
    paragraph.text = trans_text_list[i]

    # 如果格式是標題2，加粗
    style = paragraph.style.name
    if style == 'Heading 2':
        paragraph.style.font.bold = True

# 儲存成新檔案
doc.save(file_name + '_trans' + '.docx')

100%|██████████| 91/91 [00:00<00:00, 1127.73it/s]


# 其他

In [ ]:
translator_raw = pipeline("translation", model='Helsinki-NLP/opus-mt-zh-en', tokenizer=model_checkpoint)